# QAOA grid search v27

In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [2]:
import qiskit
from qiskit import Aer
from qiskit.providers.aer import AerSimulator
#from qiskit.algorithms import QAOA
from qiskit.aqua.algorithms import QAOA
from qiskit.algorithms.optimizers import SPSA
from qiskit.circuit.library import RealAmplitudes,TwoLocal
#from qiskit.utils import QuantumInstance
from qiskit.aqua import QuantumInstance

from docplex.mp.model import Model
from qiskit_optimization import QuadraticProgram
from qiskit import IBMQ
from qiskit.optimization.applications.ising import docplex
#

import logging
import numpy as np
import pandas as pd
import math


C:\Users\Maxine Khumalo\anaconda3\lib\site-packages\qiskit\optimization\__init__.py:92: DeprecationWarning: The package qiskit.optimization is deprecated. It was moved/refactored to qiskit_optimization (pip install qiskit-optimization). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('optimization', 'qiskit_optimization', 'qiskit-optimization')


Important prelim functions

In [3]:
def CSVtoNumpyArray(rawdata):
    """
    Input: 
    rawdata = a csv file (insert name as a string)

    Output:
    two numpy matrices in a tuple
    """
    data = pd.read_csv(rawdata)  #Reads the data in as a pandas object
    c = data.columns
    column = int(c[0])
    final_data1 = data.iloc[:column,:].values  #Sets data into a series of numpy arrays of strings
    final_data2 = data.iloc[column:,:].values  #1 is for the first matrix(loc) and 2 is for the second(flow)
    

    #Forms the matrix as a numpy array (easier to work with) instead of an list of lists of strings
    def string_to_integers(final_data):
        matrix = []
        for j in range(column):
            string = final_data[j][0]
            string2 = string.split(" ")
            emptyarray = []
            for i in string2:
                if i != '':
                    emptyarray.append(int(i))
            matrix.append(emptyarray)
        npmatrix = np.array(matrix) 
        return npmatrix
    return string_to_integers(final_data1),string_to_integers(final_data2)

In [4]:
def qap_value(z, MatrixLoc, MatrixFlow):
    """Compute the TSP value of a solution.
    Args:
        z (list[int]): list of allocations
        MatrixLoc (numpy array): matrix of distances
        MatrixFlow (numpy array): matrix of flow
    Returns:
        float: value of the QAP
    """
    matrix_length = len(MatrixLoc)
    x = np.reshape(z, (matrix_length,matrix_length)).astype(int)
    
    total = 0
    for i in range(matrix_length):
        for j in range(matrix_length):
            for k in range(matrix_length):
                for p in range(matrix_length):
                        total += MatrixLoc[i,j]* MatrixFlow[k,p]*x[i,k]*x[j,p]
    
    return total

Feasibility functions

In [5]:
def qap_feasible(x):
    """Check whether a solution is feasible or not.

    Args:
        x (numpy.ndarray) : binary string as numpy array.

    Returns:
        bool: feasible or not.
    """
    n = int(np.sqrt(len(x)))
    y = np.reshape(x, (n,n)).astype(int)
   
    for i in range(n):
        if sum(y[i, p] for p in range(n)) != 1:
            return False
    for p__ in range(n):
        if sum(y[i, p__] for i in range(n)) != 1:
            return False
    return True

In [6]:
def choose_best_feasible(eigenstates):
    """
    Input:
    eigenstates = dictionary
    
    Output:
    feasible binary 1D numpyarray
    probability of this answer
    
    """
    bestinarray = sorted(eigenstates.items(), key=lambda item: item[1])[::-1]
    feasible = False
    counter = 0
    total = sum(eigenstates.values())
    
    feasible=False
    while feasible==False and counter<len(bestinarray):
        #string to array
        bestasint = np.array([0])
        for i in bestinarray[counter][0]:
            bestasint = np.hstack((bestasint, int(i)))
        feasible = qap_feasible(bestasint[1:])
        frequency = bestinarray[counter][1]
        counter += 1
        
    if feasible == False:
        return feasible
    else:
        return bestasint[1:], frequency/total

## IBM device loading

In [8]:
from qiskit import IBMQ
#IBMQ.save_account('3cd4df76e664d15c0a0dde90b7ce30dd1c5f0040004c5b46c8f6055623a104b2cc23d8473c156efa1dfc6c12fbb052c1c5b25a7b9f2ad26585ebb1a93f2be810')

In [9]:
from qiskit import IBMQ

# Load account from disk
IBMQ.load_account()
IBMQ.providers()

[<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>,
 <AccountProvider for IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <AccountProvider for IBMQ(hub='ibm-q-wits', group='internal', project='wits-eie')>,
 <AccountProvider for IBMQ(hub='ibm-q-wits', group='reservations', project='reservations')>]

In [10]:
provider = IBMQ.get_provider(hub='ibm-q-wits',project='physics')
provider.backends()

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmqx2') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_armonk') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_montreal') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_toronto') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_santiago') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_bogota') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_manhattan') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_casablanca') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_sydney') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_mum

## The QAOA function

In [11]:
def testing_quantum(ins, backend, optimal_point, trials):
    """
    Input:
    ins: name of instance
    backend
    optimal_point
    trials
    
    Output:
    outmatrix : [soln,result time,eigenvector,number of spsa trials]
    """
    
    #get matrix
    datamatrix = CSVtoNumpyArray(ins)
    MatrixLoc = datamatrix[0]
    MatrixFlow = datamatrix[1]
    n = len(MatrixLoc)

    # Create an instance of a model and variables.
    thename = "qap" + str(n)
    mdl = Model(name=thename)
    x = {(i,p): mdl.binary_var(name='x_{0}_{1}'.format(i,p)) for i in range(n) for p in range(n)}

    # Object function
    qap_func = mdl.sum(MatrixLoc[i,j]* MatrixFlow[k,p]*x[i,k]*x[j,p] for i in range(n) for j in range(n) for p in range(n) for k in range(n))
    mdl.minimize(qap_func)

    # Constraints
    for i in range(n):
        mdl.add_constraint(mdl.sum(x[(i,p)] for p in range(n)) == 1)
    for p in range(n):
        mdl.add_constraint(mdl.sum(x[(i,p)] for i in range(n)) == 1)
    print(mdl.export_to_string())
    qubitOp_docplex, offset_docplex = docplex.get_operator(mdl) #potential error

    #QAOA
    machinesolutions = []
    rmachinesolutions = []
    seed = 10598
    mod = QuadraticProgram()
    mod.from_docplex(mdl)
    #print(mod.export_as_lp_string())

    #30 trials
    ans = np.zeros(5)
    for i in [1,10,100,1000]:
    #try:
        file = open("thirty_trials-" + str(backend) + str(i) + "-" + str(ins) ,"w")
        file.write("value,   feasible,   frequency, time, iteration" + "\n")
        spsa = SPSA(maxiter=i)
        Backend = AerSimulator(method='matrix_product_state')
        #Backend = provider.get_backend(backend)
        quantum_instance = QuantumInstance(backend=Backend,seed_simulator=seed,
                                           seed_transpiler=seed, skip_qobj_validation = False, shots = 8192)

        qaoa = QAOA(qubitOp_docplex,optimizer=spsa, p=1,include_custom=True)#,initial_point = optimal_point)
        result = qaoa.run(quantum_instance)
        print(result['optimal_point'])
        #print('QAOA vector:', result['eigenstate'])
        print('QAOA time:', result['optimizer_time'])
        print('QAOA QAP objective:', result['eigenvalue'] + offset_docplex)

        n = len(list(result['eigenstate'].values()))
        solution = np.hstack((np.array(list(result['eigenstate'].values())).reshape(n,1),np.array(list(result['eigenstate'].keys())).reshape(n,1)))
        print(solution)
        for r in solution:
            file.write(str(qap_value(np.array(list(r[1])),MatrixFlow,MatrixLoc)) + "," + str(qap_feasible(np.array(list(r[1])))) + "," + str(r[0]) + ","+ str(result['optimizer_time']) + "," + str(i) + "\n")
            ans = np.vstack((ans,np.array([qap_value(np.array(list(r[1])),MatrixFlow,MatrixLoc),qap_feasible(np.array(list(r[1]))),float(r[0]),result['optimizer_time'],i])))
        #except:
            #file.write("An error occurred on iteration " + str(i) + " of the 30 trials")
            #print("AN ERROR OCCURRED ON ITERATION" + str(i) + " of the 30 trials")
            #ans = np.vstack((ans,np.array([math.inf,False,0,np.nan,i])))
        print("Iteration "+ str(i) + " is complete.")
        file.close()
    return ans[1:] 
    

In [12]:
def read_optimal(ins):
    data = pd.read_csv("initial_point-" + str(ins),header = None)
    n = len(data[0]) - 1
    ans = []
    data[0][0] = data[0][0][1:]
    data[0][n] = data[0][n][:-1]
    for i in data[0]:
        r = i.split(" ")
        for t in r:
            if t!='':
                ans.append(float(t))
    return np.array(ans)

## The results cells

In [14]:
ins = "made3.csv"
import logging
logging.basicConfig(level=logging.DEBUG) # log the steps of the algorithm and results
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
backend = 'ibmq_qasm_simulator'
optimal_point = 'test'#read_optimal(ins)
ans = testing_quantum(ins, backend, optimal_point, 30)

INFO:qiskit.aqua.quantum_instance:
Qiskit Terra version: 0.17.4
Backend: 'aer_simulator_matrix_product_state (None)', with following setting:
{'basis_gates': ['ccx', 'cp', 'cswap', 'csx', 'cx', 'cy', 'cz', 'delay', 'diagonal', 'h', 'id', 'initialize', 'p', 'r', 'roerror', 'rx', 'rxx', 'ry', 'ryy', 'rz', 'rzx', 'rzz', 's', 'sdg', 'swap', 'sx', 't', 'tdg', 'u', 'u1', 'u2', 'u3', 'unitary', 'x', 'y', 'z', 'kraus', 'roerror', 'save_amplitudes', 'save_amplitudes_sq', 'save_density_matrix', 'save_expval', 'save_expval_var', 'save_matrix_product_state', 'save_probabilities', 'save_probabilities_dict', 'save_state', 'save_statevector', 'set_matrix_product_state', 'snapshot'], 'coupling_map': None}
{'initial_layout': None, 'seed_transpiler': 10598, 'optimization_level': None}
RunConfig(max_credits=10, seed_simulator=10598, shots=8192)
{'timeout': None}
{}
{}
Measurement mitigation: None
INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:
==================== Setting of QAOA =================

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: qap3

Minimize
 obj: [ 4704 x_0_0*x_1_1 + 3444 x_0_0*x_1_2 + 21280 x_0_0*x_2_1
      + 15580 x_0_0*x_2_2 + 4704 x_0_1*x_1_0 + 504 x_0_1*x_1_2
      + 21280 x_0_1*x_2_0 + 2280 x_0_1*x_2_2 + 3444 x_0_2*x_1_0
      + 504 x_0_2*x_1_1 + 15580 x_0_2*x_2_0 + 2280 x_0_2*x_2_1
      + 18368 x_1_0*x_2_1 + 13448 x_1_0*x_2_2 + 18368 x_1_1*x_2_0
      + 1968 x_1_1*x_2_2 + 13448 x_1_2*x_2_0 + 1968 x_1_2*x_2_1 ]/2
Subject To
 c1: x_0_0 + x_0_1 + x_0_2 = 1
 c2: x_1_0 + x_1_1 + x_1_2 = 1
 c3: x_2_0 + x_2_1 + x_2_2 = 1
 c4: x_0_0 + x_1_0 + x_2_0 = 1
 c5: x_0_1 + x_1_1 + x_2_1 = 1
 c6: x_0_2 + x_1_2 + x_2_2 = 1

Bounds
 0 <= x_0_0 <= 1
 0 <= x_0_1 <= 1
 0 <= x_0_2 <= 1
 0 <= x_1_0 <= 1
 0 <= x_1_1 <= 1
 0 <= x_1_2 <= 1
 0 <= x_2_0 <= 1
 0 <= x_2_1 <= 1
 0 <= x_2_2 <= 1

Binaries
 x_0_0 x_0_1 x_0_2 x_1_0 x_1_1 x_1_2 x_2_0 x_2_1 x_2_2
End



INFO:qiskit.transpiler.runningpassmanager:Pass: UnrollCustomDefinitions - 0.00000 (ms)
INFO:qiskit.transpiler.passes.basis.basis_translator:Begin BasisTranslator from source basis {('h', 1), ('measure', 1), ('rz', 1), ('rx', 1), ('cx', 2)} to target basis {'cx', 'save_probabilities', 'p', 'unitary', 'save_state', 'delay', 'save_expval', 'save_statevector', 'measure', 'tdg', 'u3', 'snapshot', 'ryy', 'initialize', 'y', 'x', 'save_probabilities_dict', 'roerror', 'u1', 'diagonal', 'sx', 'rzz', 'u', 'ry', 'rzx', 'save_matrix_product_state', 'reset', 'cp', 'rxx', 'rx', 'barrier', 'csx', 'rz', 'cswap', 'save_amplitudes_sq', 't', 'save_amplitudes', 'u2', 's', 'set_matrix_product_state', 'sdg', 'swap', 'cz', 'save_density_matrix', 'ccx', 'z', 'save_expval_var', 'kraus', 'r', 'h', 'cy', 'id'}.
DEBUG:qiskit.transpiler.passes.basis.basis_translator:Begining basis search from frozenset({('h', 1), ('measure', 1), ('rz', 1), ('rx', 1), ('cx', 2)}) to frozenset({'cx', 'save_probabilities', 'p', 'unita

INFO:qiskit.compiler.assembler:Total Assembly Time - 0.00000 (ms)
INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Energy evaluation returned [48288.86889648] - 2039.73794 (ms), eval count: 3
DEBUG:qiskit.aqua.operators.converters.circuit_sampler:Parameter binding 5.99337 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.00000 (ms)
INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Energy evaluation returned [5412.41748047] - 2117.69390 (ms), eval count: 4
DEBUG:qiskit.aqua.operators.converters.circuit_sampler:Parameter binding 4.99678 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.00000 (ms)
INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Energy evaluation returned [15662.83544922] - 2016.75963 (ms), eval count: 5
DEBUG:qiskit.aqua.operators.converters.circuit_sampler:Parameter binding 4.99630 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.00000 (ms)
INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Energy evaluation returned [2153

INFO:qiskit.compiler.assembler:Total Assembly Time - 0.00000 (ms)
INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Energy evaluation returned [48288.86889648] - 2036.74769 (ms), eval count: 32
DEBUG:qiskit.aqua.operators.converters.circuit_sampler:Parameter binding 4.99701 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.00000 (ms)
INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Energy evaluation returned [48288.86889648] - 2028.74899 (ms), eval count: 33
DEBUG:qiskit.aqua.operators.converters.circuit_sampler:Parameter binding 5.99647 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.00000 (ms)
INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Energy evaluation returned [5412.41748047] - 2051.73850 (ms), eval count: 34
DEBUG:qiskit.aqua.operators.converters.circuit_sampler:Parameter binding 5.99599 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.00000 (ms)
INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Energy evaluation returned [2

...

INFO:qiskit.compiler.assembler:Total Assembly Time - 0.00000 (ms)
INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Energy evaluation returned [2059.91040039] - 2305.58014 (ms), eval count: 1995
DEBUG:qiskit.aqua.operators.converters.circuit_sampler:Parameter binding 9.99379 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.00000 (ms)
INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Energy evaluation returned [5538.46313477] - 2218.63413 (ms), eval count: 1996
INFO:qiskit.algorithms.optimizers.spsa:Iteration 973/1001 done in 4.526212692260742.
DEBUG:qiskit.aqua.operators.converters.circuit_sampler:Parameter binding 11.99412 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.00000 (ms)
INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Energy evaluation returned [-980.52783203] - 2234.62558 (ms), eval count: 1997
DEBUG:qiskit.aqua.operators.converters.circuit_sampler:Parameter binding 9.99379 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.0000

INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Energy evaluation returned [1726.61035156] - 2114.69698 (ms), eval count: 2020
INFO:qiskit.algorithms.optimizers.spsa:Iteration 985/1001 done in 4.300352573394775.
DEBUG:qiskit.aqua.operators.converters.circuit_sampler:Parameter binding 12.99238 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.00000 (ms)
INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Energy evaluation returned [-12447.07714844] - 2110.69989 (ms), eval count: 2021
DEBUG:qiskit.aqua.operators.converters.circuit_sampler:Parameter binding 10.99324 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.00000 (ms)
INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Energy evaluation returned [5517.75830078] - 2121.69337 (ms), eval count: 2022
INFO:qiskit.algorithms.optimizers.spsa:Iteration 986/1001 done in 4.233393430709839.
DEBUG:qiskit.aqua.operators.converters.circuit_sampler:Parameter binding 11.98411 (ms)
INFO:qiskit.compiler.assembler:Total

INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Energy evaluation returned [4854.51806641] - 2162.66823 (ms), eval count: 2045
DEBUG:qiskit.aqua.operators.converters.circuit_sampler:Parameter binding 9.99331 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.00000 (ms)
INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Energy evaluation returned [-877.92236328] - 2157.67074 (ms), eval count: 2046
INFO:qiskit.algorithms.optimizers.spsa:Iteration 998/1001 done in 4.323337078094482.
DEBUG:qiskit.aqua.operators.converters.circuit_sampler:Parameter binding 13.99183 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.00000 (ms)
INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Energy evaluation returned [1989.03417969] - 2118.69717 (ms), eval count: 2047
DEBUG:qiskit.aqua.operators.converters.circuit_sampler:Parameter binding 11.99341 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.00000 (ms)
INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:Ener

DEBUG:qiskit.transpiler.passes.basis.basis_translator:Transformation path:
INFO:qiskit.transpiler.passes.basis.basis_translator:Basis translation path search completed in 0.001s.
INFO:qiskit.transpiler.passes.basis.basis_translator:Basis translation paths composed in 0.002s.
INFO:qiskit.transpiler.passes.basis.basis_translator:Basis translation instructions replaced in 0.000s.
INFO:qiskit.transpiler.runningpassmanager:Pass: BasisTranslator - 4.99940 (ms)
INFO:qiskit.transpiler.runningpassmanager:Pass: TimeUnitConversion - 2.99788 (ms)
INFO:qiskit.compiler.transpiler:Total Transpile Time - 38.97762 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.00000 (ms)


[1.47732359 7.80494623]
QAOA time: 4416.467569828033
QAOA QAP objective: (516032.3806152344+0j)
[['12' '111010010']
 ['27' '111011011']
 ['27' '110110000']
 ...
 ['8' '111010101']
 ['7' '110011100']
 ['11' '111111100']]
Iteration 1000 is complete.


In [ ]:
ins = "made4.csv"
import logging
logging.basicConfig(level=logging.DEBUG) # log the steps of the algorithm and results
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
backend ='ibmq_qasm_simulator'
optimal_point = True #read_optimal(ins)
ans = testing_quantum(ins, backend, optimal_point, 30)

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: qap4

Minimize
 obj: [ 264 x_0_0*x_1_1 + 176 x_0_0*x_1_3 + 636 x_0_0*x_2_1 + 424 x_0_0*x_2_3
      + 636 x_0_0*x_3_1 + 424 x_0_0*x_3_3 + 264 x_0_1*x_1_0 + 88 x_0_1*x_1_3
      + 636 x_0_1*x_2_0 + 212 x_0_1*x_2_3 + 636 x_0_1*x_3_0 + 212 x_0_1*x_3_3
      + 352 x_0_2*x_1_3 + 848 x_0_2*x_2_3 + 848 x_0_2*x_3_3 + 176 x_0_3*x_1_0
      + 88 x_0_3*x_1_1 + 352 x_0_3*x_1_2 + 424 x_0_3*x_2_0 + 212 x_0_3*x_2_1
      + 848 x_0_3*x_2_2 + 424 x_0_3*x_3_0 + 212 x_0_3*x_3_1 + 848 x_0_3*x_3_2
      + 480 x_1_0*x_2_1 + 320 x_1_0*x_2_3 + 744 x_1_0*x_3_1 + 496 x_1_0*x_3_3
      + 480 x_1_1*x_2_0 + 160 x_1_1*x_2_3 + 744 x_1_1*x_3_0 + 248 x_1_1*x_3_3
      + 640 x_1_2*x_2_3 + 992 x_1_2*x_3_3 + 320 x_1_3*x_2_0 + 160 x_1_3*x_2_1
      + 640 x_1_3*x_2_2 + 496 x_1_3*x_3_0 + 248 x_1_3*x_3_1 + 992 x_1_3*x_3_2
      + 660 x_2_0*x_3_1 + 440 x_2_0*x_3_3 + 660 x_2_1*x_3_0 + 220 x_2_1*x_3_3
      + 880 x_2_2*x_3_3 + 440 x_2_3*x_3_0 + 220 x_

INFO:qiskit.aqua.quantum_instance:
Qiskit Terra version: 0.17.4
Backend: 'aer_simulator_matrix_product_state (None)', with following setting:
{'basis_gates': ['ccx', 'cp', 'cswap', 'csx', 'cx', 'cy', 'cz', 'delay', 'diagonal', 'h', 'id', 'initialize', 'p', 'r', 'roerror', 'rx', 'rxx', 'ry', 'ryy', 'rz', 'rzx', 'rzz', 's', 'sdg', 'swap', 'sx', 't', 'tdg', 'u', 'u1', 'u2', 'u3', 'unitary', 'x', 'y', 'z', 'kraus', 'roerror', 'save_amplitudes', 'save_amplitudes_sq', 'save_density_matrix', 'save_expval', 'save_expval_var', 'save_matrix_product_state', 'save_probabilities', 'save_probabilities_dict', 'save_state', 'save_statevector', 'set_matrix_product_state', 'snapshot'], 'coupling_map': None}
{'initial_layout': None, 'seed_transpiler': 10598, 'optimization_level': None}
RunConfig(max_credits=10, seed_simulator=10598, shots=8192)
{'timeout': None}
{}
{}
Measurement mitigation: None
INFO:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:
==================== Setting of QAOA =================

INFO:qiskit.aqua.algorithms.vq_algorithm:Starting optimizer.
bounds=[(None, None), (0, 6.283185307179586)]
initial point=[0.27844089 2.58019693]
INFO:qiskit.transpiler.runningpassmanager:Pass: UnrollCustomDefinitions - 0.99945 (ms)
INFO:qiskit.transpiler.passes.basis.basis_translator:Begin BasisTranslator from source basis {('h', 1), ('measure', 1), ('rz', 1), ('rx', 1), ('cx', 2)} to target basis {'cx', 'save_probabilities', 'p', 'unitary', 'save_state', 'delay', 'save_expval', 'save_statevector', 'measure', 'tdg', 'u3', 'snapshot', 'ryy', 'initialize', 'y', 'x', 'save_probabilities_dict', 'roerror', 'u1', 'diagonal', 'sx', 'rzz', 'u', 'ry', 'rzx', 'save_matrix_product_state', 'reset', 'cp', 'rxx', 'rx', 'barrier', 'csx', 'rz', 'cswap', 'save_amplitudes_sq', 't', 'save_amplitudes', 'u2', 's', 'set_matrix_product_state', 'sdg', 'swap', 'cz', 'save_density_matrix', 'ccx', 'z', 'save_expval_var', 'kraus', 'r', 'h', 'cy', 'id'}.
DEBUG:qiskit.transpiler.passes.basis.basis_translator:Begini

In [ ]:
ins = "made6.csv"
import logging
logging.basicConfig(level=logging.DEBUG) # log the steps of the algorithm and results
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
backend ='ibmq_qasm_simulator'
optimal_point = 'test'
#optimal_point = read_optimal(ins)
ans = testing_quantum(ins, backend, optimal_point, 30)

In [ ]:
ins = "made7.csv"
import logging
logging.basicConfig(level=logging.DEBUG) # log the steps of the algorithm and results
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
backend ='ibmq_qasm_simulator'
#optimal_point = read_optimal(ins)
ans = testing_quantum(ins, backend, optimal_point, 30)

In [ ]:
ins = "made8.csv"
import logging
logging.basicConfig(level=logging.DEBUG) # log the steps of the algorithm and results
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
backend ='ibmq_qasm_simulator'
#optimal_point = read_optimal(ins)
ans = testing_quantum(ins, backend, optimal_point, 30)

In [ ]:
ins = "made5.csv"
import logging
logging.basicConfig(level=logging.DEBUG) # log the steps of the algorithm and results
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
backend ='ibmq_qasm_simulator'
optimal_point = True#read_optimal(ins)
ans = testing_quantum(ins, backend, optimal_point, 30)

In [ ]:
"""runtime"""

In [ ]:
qiskit.__qiskit_version__